In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_csv("InputData.csv")

In [3]:
import pandas as pd


def disaggregateInputData(df):
    dfPrices = df.iloc[:, 0:6]
    dfDurations = df.iloc[:, [0, 6, 7, 8, 9, 10]]
    dfOptWeights = df.iloc[:, [0, 11, 12, 13, 14, 15]]
    dfRollingStats = df.loc[:, ["DateTime", "RollingAvg", "RollingStd"]]

    dfPrices.set_index("DateTime", inplace=True)
    dfDurations.set_index("DateTime", inplace=True)
    dfOptWeights.set_index("DateTime", inplace=True)
    dfRollingStats.set_index("DateTime", inplace=True)

    dfPrices.index = pd.to_datetime(dfPrices.index)
    dfDurations.index = pd.to_datetime(dfDurations.index)
    dfOptWeights.index = pd.to_datetime(dfOptWeights.index)
    dfRollingStats.index = pd.to_datetime(dfRollingStats.index)
    return (dfPrices, dfDurations, dfOptWeights, dfRollingStats)

In [100]:
class Positions:
    def __init__(self, positions, startIdx, endIdx):
        self.positions = np.array(positions)
        self.startIdx = startIdx
        self.endIdx = endIdx

    def duration(self):
        return self.endIdx - self.startIdx


class PortPositions:
    def __init__(self, timeSize, numEquities):
        # print(timeSize)
        self.startIdx = 0
        self.endIdx = timeSize - 1
        self.cumPositions = np.zeros((timeSize, numEquities))
        self.numPositions = np.zeros(timeSize)

    # todo: i in range number to number
    # i in df.index[]
    def addPositions(self, positionsChange):
        for i in range(positionsChange.startIdx, positionsChange.endIdx):
            #             print("##################################################")
            #             print(type(positionsChange.positions))
            #             print(type(self.cumPositions[i,:]))
            self.cumPositions[i, :] = self.cumPositions[i, :] + positionsChange.positions
            self.numPositions[i] = self.numPositions[i] + 1

    def maxPosJudge(self, positionsChange, maxPos):
        startIdx = positionsChange.startIdx
        # print(startIdx)
        # print(self.numPositions[startIdx])
        return self.numPositions[startIdx] < maxPos

    def get_cumPositions():
        return self.cumPositions

    def get_numPositions():
        return self.numPositions


class BackTestingSystem:


    def __init__(self, numEquities, pointPrices, tickSizePrices, margins, transactionCostCoeff):
        self.numEquities = numEquities
        if (len(pointPrices) == numEquities):
            self.pointPrices = np.array(pointPrices)
        else:
            print("number of equities unmatch: point prices")
        if (len(tickSizePrices) == numEquities):
            self.tickSizePrices = np.array(tickSizePrices)
        else:
            print("number of equities unmatch: tickSizes")
        if (len(margins) == numEquities):
            self.margins = np.array(margins)
        else:
            print("number of equities unmatch: margins")
            
        self.transactionCostCoeff = transactionCostCoeff
        # 

    def set_rollDate(self, rollDate):
        self.rollDate = rollDate

    def get_rollDate(self):
        return self.rollDate

    def set_exitUpLevel(self, exitUpLevel):
        self.exitUpLevel = exitUpLevel

    def set_exitDownLevel(self, exitDownLevel):
        self.exitDownLevel

    def set_triggerS(self, triggerS):
        self.triggerS = triggerS

    def set_triggerT(self, triggerT):
        self.triggerT = triggerT

    def get_marginPrices(self):
        return self.margins / self.pointPrices

    def get_tickSizes(self):
        return self.pointPrices * tickSizePrices

    def set_AUM(self, AUM):
        self.AUM = AUM

    def set_rollingStats(self, dfRollingStats):
        self.dfRollingStats = dfRollingStats
        self.df = pd.concat([self.df, self.rollingStats], axis=1)

    def set_maxPoistions(self, maxPositions):
        self.maxPositions = 30

    def set_percentageInvested(self, pctInvest):
        self.percentageInvested = pctInvest

    def set_maxPositions(self, maxPositions):
        self.maxPositions = maxPositions

    def set_exitUpLevel(self, exitUpLevel):
        self.exitUpLevel = exitUpLevel

    def set_exitDownLevel(self, exitDownLevel):
        self.exitDownLevel = exitDownLevel

    def input_data(self, dfPrices, dfDurations, dfOptWeights, dfRollingStats):
        self.dfPrices = dfPrices
        self.dfDurations = dfDurations
        self.dfOptWeights = dfOptWeights
        self.df = pd.concat([self.dfPrices, self.dfDurations, self.dfOptWeights, dfRollingStats], axis=1)

    # todo: delete
    #     def input_whole_data(self,df):
    #         self.df = df

    def get_df(self):
        return self.df

    def time_delta_365(self, timeDelta):
        if (timeDelta.days > 0):
            return timeDelta.days / 365
        else:
            return 0

    def preprocessing(self):

        print("****************************************************************")
        print("Start preprocessing...")
        # basic setting
        self.marginPrices = self.margins / self.pointPrices
        self.maxInitMargin = self.AUM * self.percentageInvested
        self.positionInitMargin = self.maxInitMargin / self.maxPositions
        self.tickSizes = self.pointPrices * self.tickSizePrices
        self.marginPrices = self.margins / self.pointPrices

        # time to maturity
        timeDeltas = self.rollDate - self.df.index
        self.df['TimeToMaturity'] = timeDeltas
        self.df.TimeToMaturity = self.df.TimeToMaturity.apply(self.time_delta_365)
        self.timeToMaturity = self.df.TimeToMaturity
        print(self.df.head())
        # future duration
        futureDurationsColumns = ["dfFutureDuration" + dur_str[8:] for dur_str in self.dfDurations.columns]
        self.dfFutureDurations = pd.DataFrame(index=self.df.index, columns=futureDurationsColumns)
        for index, row in self.dfDurations.iterrows():
            self.dfFutureDurations.loc[index, :] = (row - self.df.TimeToMaturity[index]).values

        # margin unit
        #         self.marginUnit = pd.Series(index = self.df.index, name="MarginUnit")
        #         for index, row in self.dfOptWeights.iterrows():
        #             self.marginUnit[index] = np.inner(np.abs(row.values), self.marginPrices)
        self.marginUnit = self.dfOptWeights.apply(lambda x: np.inner(np.abs(x), self.marginPrices), axis=1)
        self.marginUnit.rename("MarginUnit")

        # national
        self.portNotional = self.positionInitMargin / self.marginUnit
        self.portNotional.rename("PortNotional", inplace=True)

        # positions
        positionsColumns = ["dfPosition" + dur_str[8:] for dur_str in self.dfDurations.columns]
        self.dfPositions = pd.DataFrame(index=self.df.index, columns=positionsColumns)
        for index, row in self.dfOptWeights.iterrows():
            self.dfPositions.loc[index, :] = row.values * self.portNotional[index] / self.pointPrices
        # tick size
        self.portTickSize = self.dfPositions.apply(lambda x: np.inner(np.abs(x), self.tickSizes), axis=1)
        self.portTickSize.rename("PortTickSize", inplace=True)

        # current price
        self.portPrice = pd.Series(index=self.df.index, name="PortPrice")
        for idx in self.df.index:
            self.portPrice[idx] = np.inner(self.dfPrices.loc[idx, :], self.dfOptWeights.loc[idx, :])

        # tick size price
        self.portTickSizePrice = pd.Series(index=self.df.index, name="PortTickSizePrice")
        for idx in self.df.index:
            self.portTickSizePrice[idx] = self.portTickSize[idx] / self.portNotional[idx]

        # z-score
        self.ZScore = pd.Series(index=self.df.index, name="ZScore")
        for idx in self.df.index:
            self.ZScore[idx] = (self.portPrice[idx] - self.df.RollingAvg[idx]) / self.df.RollingStd[idx]

        # t-score
        self.TScore = pd.Series(index=self.df.index, name="TScore")
        for idx in self.df.index:
            self.TScore[idx] = (self.portPrice[idx] - self.df.RollingAvg[idx]) / self.portTickSizePrice[idx]

        # concat all results
        self.df = pd.concat([self.df, self.dfFutureDurations, self.marginUnit, self.portNotional,
                             self.dfPositions, self.portTickSize, self.portPrice, self.portTickSizePrice,
                             self.ZScore, self.TScore], axis=1)

        print("Preprocessing finished!")
        print("****************************************************************")
        return self.df

    def _enterSignal(self, time):
        return self.ZScore[time] <= -self.triggerS and self.TScore[time] <= -self.triggerT and time < self.rollDate

    def _exitTime(self, startTime,rollTime=None):
        positions = self.dfPositions.loc[startTime, :]
        p0 = np.sum(positions.values * self.dfPrices.loc[startTime, :].values * self.pointPrices)
        exitUp = self.exitUpLevel * self.portTickSize[startTime]
        exitDown = self.exitDownLevel * self.portTickSize[startTime]
        startIdx = self.df.index.get_loc(startTime)
        for time in self.df.index[startIdx:]:
            price = np.sum(positions.values * self.dfPrices.loc[time, :].values * self.pointPrices)
            #             print(price)
            #             print(p0)
            if (price - p0 >= exitUp):
                break
            if (price - p0 <= -exitDown):
                break
            #         print(time)
        if(rollTime and time>rollTime):
            time = rollTime
            
        return time

    # todo: change misleading name upwards, "port" is the term for portfolio, if not execute, call "df"
    def calculateCumPositions(self):
        print("**************************************************")
        print("start calculate strategy positions")
        self.portPositions = PortPositions(len(self.df.index), self.numEquities)
        for idx, time in enumerate(self.df.index):
            positions = self.dfPositions.iloc[idx, :]
            endTimeIdx = self.df.index.get_loc(self._exitTime(time,self.rollDate))
            positionsChange = Positions(positions, idx, endTimeIdx)
            #             print("positionsChange.positions",positionsChange.positions)
            if (self._enterSignal(time) and self.portPositions.maxPosJudge(positionsChange, self.maxPositions)):
                self.portPositions.addPositions(positionsChange)
                print("##########################################")
                print("add positions:", positionsChange.positions)
                print("time:",time)
                print("number of positions:",self.portPositions.numPositions[idx])
                print("startTime:",self.df.index[positionsChange.startIdx])
                print("endIdx:",self.df.index[positionsChange.endIdx])
                print("cumPositions:",self.portPositions.cumPositions[idx])
                print("##########################################")
        print("complete calculation")
        print("**************************************************")
        return self.portPositions
    
    def calculateInitMargin(self):
        if(not self.portPositions):
            calculateCumPositions()
        
        self.portInitMargin = np.inner(np.abs(self.portPositions.cumPositions),self.margins)
        return self.portInitMargin
    
    def calculatePnL(self):
        self.PnL = [0]
        for idx,time in enumerate(self.df.index[1:]):
            self.PnL.append(np.sum(self.pointPrices*self.portPositions.cumPositions[idx-1]
                            *(self.dfPrices[idx] - self.dfPrices[idx-1])))
        self.PnL = np.array(self.PnL)
        return self.PnL
    
    def calculateTransactionCost(self):
        self.transactionCost = [0]
        for idx,time in enumerate(self.df.index[1:]):
            self.transactionCost.append(np.inner(
                np.abs(self.portPositions.cumPositions[idx] - self.portPositions.cumPositions[idx-1]),
                self.tickSizes)*self.transactionCostCoeff)
        self.transactionCost = np.array(self.transactionCost)
        return self.transactionCost
    
    def calculateNetPnL(self):
        if(not self.PnL):
            calculatePnL()
        if(not self.transactionCost):
            calculateTransactionCost()
        self.netPnL = self.PnL - self.transactionCost
        return self.netPnL
            


In [102]:
import pandas as pd
import numpy as np
# from datetime import datetime
# from BackTesting import BackTestingSystem
# from utils import disaggregateInputData

df = pd.read_csv("InputData.csv")

# input data
(dfPrices,dfDurations,dfOptWeights,dfRollingStats) = disaggregateInputData(df)

# print(rollingStats.head())
pointPrices = [2000,1000,1000,1000,1000]
tickSizePrices = [1/128,1/128,1/64,1/32,1/32]
margins = [380,625,1300,2700,3700]
transactionCostCoeff = 0.5
# hyper parameteres
numEquities = 5
AUM = 10000000

# model parameters
rollDate = datetime(2017,3,1)
triggerS = 2
triggerT = 5
exitUpLevel = 2
exitDownLevel = 20
pctInvested = 0.3
maxPositions = 30

# plug in
# HyperParameteres
backTesting = BackTestingSystem(numEquities,pointPrices,tickSizePrices,margins,transactionCostCoeff)
# Model Parameters
backTesting.set_AUM(AUM)
backTesting.set_percentageInvested(pctInvested)
backTesting.set_maxPoistions(maxPositions)
backTesting.set_rollDate(rollDate)
backTesting.set_triggerS(triggerS)
backTesting.set_triggerT(triggerT)
backTesting.set_exitUpLevel(exitUpLevel)
backTesting.set_exitDownLevel(exitDownLevel)
# History Data
backTesting.input_data(dfPrices,dfDurations,dfOptWeights,dfRollingStats)

# pre-processing
df2 = backTesting.preprocessing()
print(df2.head())

# strategy's cumulative positions
portCumPositions = backTesting.calculateCumPositions()
portInitMargin = backTesting.calculateInitMargin()
portNetPnL = backTesting.calculateNetPnL()

****************************************************************
Start preprocessing...
                        PriceTU     PriceFV     PriceTY    PriceUS    PriceUB  \
DateTime                                                                        
2017-01-02 19:00:00  108.296875  117.523438  124.046875  150.21875  159.59375   
2017-01-02 20:00:00  108.296875  117.539062  124.078125  150.31250  159.75000   
2017-01-02 21:00:00  108.304688  117.554688  124.093750  150.34375  159.75000   
2017-01-02 22:00:00  108.312500  117.578125  124.109375  150.34375  159.78125   
2017-01-02 23:00:00  108.312500  117.585938  124.125000  150.43750  159.90625   

                     DurationTU  DurationFV  DurationTY  DurationUS  \
DateTime                                                              
2017-01-02 19:00:00    1.957515     4.24503    6.222844    13.85058   
2017-01-02 20:00:00    1.957515     4.24503    6.222844    13.85058   
2017-01-02 21:00:00    1.957515     4.24503    6.222844    1

AttributeError: 'BackTestingSystem' object has no attribute 'PnL'

In [103]:
backTesting.calculatePnL()

KeyError: 0

In [97]:
backTesting.dfPrices.iloc[1]

PriceTU    108.296875
PriceFV    117.539062
PriceTY    124.078125
PriceUS    150.312500
PriceUB    159.750000
Name: 2017-01-02 20:00:00, dtype: float64

In [99]:
np.sum(backTesting.pointPrices*backTesting.portPositions.cumPositions[0]*(backTesting.dfPrices.iloc[1] - backTesting.dfPrices.iloc[0]))

0.0

In [64]:
np.inner([1,2,3],[[1,2,3],[4,5,6]])

array([14, 32])

In [68]:
np.inner(margins,np.abs(portCumPositions.cumPositions))[0:100]

array([      0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
             0.,       0.,       0.,       0.,       0.,       0.,
        100000.,  100000.,  100000.,  100000.,  100000.,  100000.,
        100000.,  100000.,  100000.,  100000.,  100000.,  100000.,
        100000.,  100000.,  200000.,  200000.,  200000.,  200000.,
        200000.,  200000.,  200000.,  200000.,  200000.,  200000.,
        200000.,  200000.,  200000.,  200000.,  200000.,  200000.,
        200000.,  200000.,  200000.,  200000.,  200000.,       0.,
             0.,       0.,       0.,       0.,       0.,      